Note: Une grande partie du code ci-dessous à été extrait ou inspiré de la ressource pour utiliser YAMNet. Crédit à https://www.tensorflow.org/hub/tutorials/yamnet et https://www.tensorflow.org/hub/tutorials/yamnet

# Préparation

In [ ]:
# Installation du bon TensorFlow
!pip install -q "tensorflow==2.11.*"
# tensorflow_io 0.28 is compatible with TensorFlow 2.11
!pip install -q "tensorflow_io==0.28.*"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 55.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.10 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
google-cloud-aiplatform 1.74.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=

In [ ]:
# Importer librairies
from google.colab import files
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

import os
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

In [ ]:
# Chargement du modèle YAMNet de Google
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [ ]:
# Fonctions utilitaires pour charger des fichiers audio et s'assurer que la fréquence d'échantillonnage est correcte
@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
print(len(os.listdir()))

2353


In [ ]:
# Chargement de la données
file_path = '/content/annotations_clean_and_noised.xlsx'
df = pd.read_excel(file_path)

In [ ]:
# Mettre les données dans le bon format (tensorflow)
filenames = df['filename']
targets = df['target']
folds = df['fold']

main_ds = tf.data.Dataset.from_tensor_slices((filenames, targets, folds))
main_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
# Transformer les fichiers audio dans le format appropriés (établie par notre fonction intérieur)
def load_wav_for_map(filename, label, fold):
  return load_wav_16k_mono(filename), label, fold

main_ds = main_ds.map(load_wav_for_map)
main_ds.element_spec

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
# Applique le modèle d'extraction d'embeddings à des données audio WAV
def extract_embedding(wav_data, label, fold):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings),
            tf.repeat(fold, num_embeddings))

# extraction d'embeddings pour chaque donnée
main_ds = main_ds.map(extract_embedding).unbatch()
main_ds.element_spec

(TensorSpec(shape=(1024,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
# Split des données et mise en cache
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold < 7)
val_ds = cached_ds.filter(lambda embedding, label, fold: fold == 7)

# supprimer la colonne fold
remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)

# mélange des données, batching, préchargement
train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Instancier les noms des nouveaux labels
my_classes = ['0', '1','2','3','4']
map_class_to_id = {'0':0, '1':1,'2':2,'3':3,'4':4}

# Entrainement

In [ ]:
# Entrée : Un vecteur de taille 1024 (représentant un embedding)
# Couche cachée : 3 couche dense avec 64 neurones et une activation ReLU
# Couche sortie : nombre de classe

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(my_classes))
], name='MLP')

model.summary()

Model: "MLP"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               131200    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                                 
Total params: 164,869
Trainable params: 164,869
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# prépare le modèle pour l'entraînement
# Crossentropyloss/ softmax pour la classfication
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

# EarlyStopping pour arrêter l'entraînement lorsque le modèle cesse d'améliorer
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5,
                                            restore_best_weights=True)

In [ ]:
# Entraînement du modèle (Suivi de l'entrainement sur x epoch, en validation)
history = model.fit(train_ds,
                       epochs=100,
                       validation_data=val_ds,
                       callbacks=callback)

Epoch 1/100
509/509 [==============================] - 153s 246ms/step - loss: 1.1110 - accuracy: 0.5237 - val_loss: 1.3221 - val_accuracy: 0.5039
Epoch 2/100
509/509 [==============================] - 2s 4ms/step - loss: 1.0732 - accuracy: 0.5295 - val_loss: 1.2916 - val_accuracy: 0.4976
Epoch 3/100
509/509 [==============================] - 3s 5ms/step - loss: 1.0613 - accuracy: 0.5368 - val_loss: 1.3651 - val_accuracy: 0.5016
Epoch 4/100
509/509 [==============================] - 2s 4ms/step - loss: 1.0511 - accuracy: 0.5412 - val_loss: 1.2652 - val_accuracy: 0.4937
Epoch 5/100
509/509 [==============================] - 2s 3ms/step - loss: 1.0425 - accuracy: 0.5425 - val_loss: 1.3323 - val_accuracy: 0.5024
Epoch 6/100
509/509 [==============================] - 2s 3ms/step - loss: 1.0310 - accuracy: 0.5471 - val_loss: 1.3666 - val_accuracy: 0.4972
Epoch 7/100
509/509 [==============================] - 2s 4ms/step - loss: 1.0219 - accuracy: 0.5528 - val_loss: 1.2806 - val_accuracy: 0.

# Sauvegarder le modèle entrainé

In [ ]:
class ReduceMeanLayer(tf.keras.layers.Layer):
  def __init__(self, axis=0, **kwargs):
    super(ReduceMeanLayer, self).__init__(**kwargs)
    self.axis = axis

  def call(self, input):
    return tf.math.reduce_mean(input, axis=self.axis)

In [ ]:
saved_model_path = './Model_on_YAMNet_clean_and_noised'

input_segment = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='audio')
embedding_extraction_layer = hub.KerasLayer(yamnet_model_handle,
                                            trainable=False, name='YAMNet')
_, embeddings_output, _ = embedding_extraction_layer(input_segment)
serving_outputs = model(embeddings_output)
serving_outputs = ReduceMeanLayer(axis=0, name='classifier')(serving_outputs)
serving_model = tf.keras.Model(input_segment, serving_outputs)
serving_model.save(saved_model_path, include_optimizer=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Téléchargement du modèle
!cp -r /content/Model_on_YAMNet_clean_and_noised /content/drive/MyDrive/